In [ ]:
!pip install pymorphy2
!pip install natasha
import re
import operator
from string import punctuation
from pymystem3 import Mystem
import re
import gensim
import logging
import nltk.data
import pandas as pd
import urllib.request
from nltk.corpus import stopwords
from gensim.models import word2vec
from pymorphy2 import MorphAnalyzer

import warnings
warnings.filterwarnings('ignore')
import collections

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

import spacy
import scipy
from string import punctuation
import pymorphy2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import random
random.seed = 23
from collections import Counter

In [ ]:
import spacy
import scipy
from string import punctuation
import pymorphy2
import sqlite3
import pandas as pd
import re
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import random
random.seed = 23

In [ ]:
import re
import gensim
import logging
import nltk.data
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from gensim.models import word2vec

import warnings
import nltk
nltk.download('stopwords')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
with open('czech_predicates.txt') as f:
    text = f.read()

In [ ]:
lines = text.splitlines() 

In [ ]:
print(lines)

['Kwic', 'Já také nevím ', 'Vyřešeno Ještě přemýšlím ', 'Prostě by z textu nemělo být poznat ', 'supr toho jsem si nevšiml ', 'Pór nebo pórek ó obsahuje ale nevím ', 'Když je třeba stáhnout nový ActiveX je uživatel dotázán ', 'To je jako kdybyste se mě zeptal ', 'Hodně se o tom spekulovalo ', 'Různé barvy a ikona visacího zámku ukazují ', 'Chtěla údajně zjistit ', 'Prosím zkontrolujte ', 'Otázka však je ', 'Potom zde sledovali ', 'No přestávám mít čas hledat dál ani nevím ', 'Nebylo ovšem řečeno ', 'Ještě uvidím ', 'Nábožensky orientovaná píseň. "Asi pět minut jsem přemýšlel ', 'Tento spoj je hlavní částí projektu který má zjistit ', 'Sledují jí a spekulují ', 'Babiš se stal předsedou ANO 2011. </s><s> Zatím váhá ', 'Zatím váhá ', 'Po prvním spuštění se vás program zeptá ', 'Nyní ministerstvo životního prostředí bude prozkoumávat ', 'Ale o to přeci vůbec nejde ', 'Jelikož mě podobná témata docela zajímají ', 'Pravidelně navštěvuje chatovací místnost na internetu a přemýšlí ', 'Člověk n

In [ ]:
targs = []
for l in lines:
  sent = l.split()
  print(sent)
  if len(sent) > 1:
    targ = sent[-1]
    targs.append(targ)

Streaming output truncated to the last 5000 lines.
['Kapitola', '2.Tu', 'se', 'přikradl', 'druhej', 'Policajt', 'aby', 'se', 'zeptal']
['Někteří', 'se', 'dívají', 'bokem', 'a', 'sledují']
['Helena', 'ŠULCOVÁ', 'moderátorka:', 'Té', 'pondělní', 'reportáže.', '</s><s>', 'Nevím']
['Vy', 'jste', 'mi', 'řekla']
['A', 'vy', 'se', 'mě', 'ptáte']
['Možná', 'by', 'ses', 'onoho', 'kluka', 'mohl', 'zeptat']
['Helena', 'ŠULCOVÁ', 'moderátorka:', 'A', 'kdy', 'se', 'rozhodnete']
['Před', 'poskytnutím', 'úvěru', 'může', 'finanční', 'instituce', 'v', 'této', 'databázi', 'ověřit']
['Otázkou', 'je']
['Protože', 'člověk', 'musí', 'vždy', 'zvažovat']
['Uživatelé', 'tohoto', 'programu', 'tak', 'budou', 'moci', 'například', 'rozpoznat']
['Uživatelé', 'tohoto', 'programu', 'tak', 'budou', 'moci', 'například', 'rozpoznat']
['Bylo', 'to', 'něco', 'neskutečného', 'protože', 'už', 'nevím']
['Já', 'jsem', 'tam', 'byla', 'někdy', 'v', 'osmém', 'měsíc', 'jen', 'se', 'zeptat']
['Paní', 'učitelka', 'se', 'ptala']
['P

In [ ]:
df = pd.read_csv("C_comp_jestli_zda_predicates.csv", sep = ",")
df

,w1,c1,r1,w2,c2,r2
0,absolvovat,199,0.01581,absolvovat,27,0.00215
1,analyzovat,125,0.00993,analyzovat,476,0.03782
2,bádat,59,0.00469,aplikovat,19,0.00151
3,bát,3075,0.24431,argumentovat,33,0.00262
4,bavit,599,0.04759,bádat,88,0.00699
...,...,...,...,...,...,...
995,zvì,33,0.00262,zvì,22,0.00175
996,zvládat,71,0.00564,zvítězit,31,0.00246
997,zvládnout,238,0.01891,zvládat,24,0.00191
998,zvolit,1460,0.11600,zvládnout,113,0.00898


In [ ]:
conn = sqlite3.connect('parataxis.db')

In [ ]:
c = conn.cursor()

In [ ]:
c.execute("CREATE TABLE IF NOT EXISTS first(word text PRIMARY KEY, counts integer,  freq integer)")

In [ ]:
for p in range(len(df["w1"].to_list())):
    x = df["w1"].to_list()[p]
    y = df["c1"].to_list()[p]
    z = df["r1"].to_list()[p]
    c.execute('INSERT INTO first VALUES (?, ?, ?)', (x, y, z))
conn.commit()

In [ ]:
c.execute("CREATE TABLE IF NOT EXISTS second(verb text PRIMARY KEY, counts_verb integer,  freq_verb integer)")

In [ ]:
for p in range(len(df["w1"].to_list())):
    x = df["w2"].to_list()[p]
    y = df["c2"].to_list()[p]
    z = df["r2"].to_list()[p]
    c.execute('INSERT INTO second VALUES (?, ?, ?)', (x, y, z))
conn.commit()

In [ ]:
new = 'SELECT * FROM first JOIN second ON first.word = second.verb'

In [ ]:
df_new = pd.read_sql_query(new, con=conn)

In [ ]:
df_new

,word,counts,freq,verb,counts_verb,freq_verb
0,absolvovat,199,0.01581,absolvovat,27,0.00215
1,analyzovat,125,0.00993,analyzovat,476,0.03782
2,bádat,59,0.00469,bádat,88,0.00699
3,bát,3075,0.24431,bát,1058,0.08406
4,bavit,599,0.04759,bavit,255,0.02026
...,...,...,...,...,...,...
761,zveřejnit,71,0.00564,zveřejnit,142,0.01128
762,zvì,33,0.00262,zvì,22,0.00175
763,zvládat,71,0.00564,zvládat,24,0.00191
764,zvládnout,238,0.01891,zvládnout,113,0.00898


In [ ]:
def difference(x,y):
  r = x/y*100
  return r

In [ ]:
df_new['diff'] = df_new.apply(lambda x: difference(x.counts, x.counts_verb), axis=1)

In [ ]:
df_new

,word,counts,freq,verb,counts_verb,freq_verb,diff
0,absolvovat,199,0.01581,absolvovat,27,0.00215,737.037037
1,analyzovat,125,0.00993,analyzovat,476,0.03782,26.260504
2,bádat,59,0.00469,bádat,88,0.00699,67.045455
3,bát,3075,0.24431,bát,1058,0.08406,290.642722
4,bavit,599,0.04759,bavit,255,0.02026,234.901961
...,...,...,...,...,...,...,...
761,zveřejnit,71,0.00564,zveřejnit,142,0.01128,50.000000
762,zvì,33,0.00262,zvì,22,0.00175,150.000000
763,zvládat,71,0.00564,zvládat,24,0.00191,295.833333
764,zvládnout,238,0.01891,zvládnout,113,0.00898,210.619469


In [ ]:
df_new.to_csv("C_comp_predicates_jesli_zda_compar1.csv", index=None)